#### [미션] 여러 page 데이터 가져오기(지식인 검색)
- 사용자로부터 검색어, 페이지 입력
- 검색어에 대한 답변 추출 => csv 형식으로 데이터 저장
- 저장내용: 검색어/등록일/답변


##### 작업순서
1. url 구성: 기본url + 검색어 + 페이지(변경)
2. 기본 URL 추출
3. 기본 URL을 이용한 답변 추출
4. DataFrame으로 변경후 검색어.csv로 저장

1. 사용자에게 검색어와 페이지 건수 입력 및 url 추출

In [1]:
import requests
import pandas as pd
import tqdm
from bs4 import BeautifulSoup as bs

In [2]:
# url 생성
sch=input('검색어 입력:')
pages=input('종료페이지 입력:')

url_list=[]

for page in tqdm.tqdm(range(1, int(pages)+1)):
    url = f'https://kin.naver.com/search/list.naver?query={sch}&page={page}'
    html = requests.get(url)
    if html.status_code != 200:continue
    soup = bs(html.text, 'html.parser') 
    lis = soup.select('ul.basic1 li')
    for li in lis:
        url_list.append(li.find('a')['href'])

print("\n",len(url_list))

검색어 입력:부산관광지
종료페이지 입력:100


100%|██████████| 100/100 [00:46<00:00,  2.15it/s]


 1000


2. 추출한 URL을 이용해 답변 추출

In [3]:
answer_list=[]

for url in tqdm.tqdm(url_list):
    # 정보 요청
    html = requests.get(url)
    if html.status_code != 200: # 정보 오류 여부 확인
        continue

    # 정보 추출
    soup = bs(html.text, 'html.parser') 
    anscnt=soup.find('em', class_="_answerCount num").text
    if int(anscnt) < 1:
        continue  
    #print(anscnt)

    # 답변 텍스트 추출
    ques_date=soup.select('div.question-content span.c-userinfo__info')[0].text
    title=soup.select('div.question-content div.title')[0].text
    try:
        question=soup.select('div.question-content div.c-heading__content')[0].text
    except:
        question="내용없음"
    answers=soup.select('div.answer-content div.se-main-container')

    # 데이터 추가
    for answer in answers:
        answer_list.append({'ques_date':ques_date.replace('작성일',""),
                        'title':title.replace('\n',"").replace('\t',""),
                        'question':question.replace('\n',"").replace('\t',""),
                        'answer':answer.text.replace('\n',"").replace('\t',"")})

100%|██████████| 1000/1000 [08:24<00:00,  1.98it/s]


3. 데이터프레임 형식으로 변경후 csv 저장

In [4]:
df=pd.DataFrame(answer_list)
df

,ques_date,title,question,answer
0,2021.03.24,부산 관광지,제가 다음주에 부산으로 여행을 가는데 가볼만한 관광지 추천좀 해주세요!!!,부산하면바다지요.부산 해운대관광열차타고 바다위를 한바퀴 쭉 돌다보면~속이 ...
1,끌올 2022.12.19,"부산 워크숍 관광지, 식당",안녕하세요~! 23년 2월에 부산으로 워크숍을 가게 됐습니다.인원은 25명 정도이고...,"호텔셔틀 여부 물어보고아니면 부산,울산소재 관광버스회사에 사전 예약하면 됩..."
2,끌올 2022.12.19,"부산 워크숍 관광지, 식당",안녕하세요~! 23년 2월에 부산으로 워크숍을 가게 됐습니다.인원은 25명 정도이고...,부산 워크샵 버스대절 알아보고 계심 여기서 견적 한번 받아보시는거 어떠세요...
3,2021.05.16,부산 관광지 소개하기,"중학교 1학년인데요 사회 수행평가로 부산의 관광지 3개, 음식 3개, 숙박1개, 교...","안녕하세요 마음을 전달하는 시바 입니다 ლ(╹◡╹ლ) ​해운대, 광안리, ..."
4,2021.03.14,부산 관광지 순위,해운대가 1등?아님광안리가 1등?뭐가 사람들이 제일 좋아하는 관광지인가요?,외국인이 한창많올때부산의관광지순위가 서면이1위 그다음 해운대 광안리 남포동...
...,...,...,...,...
1382,2020.07.20,부산 여행코스,10시 조금넘어서 부산역에 도착해서 가까운곳 먼저갈려는데 유명하고 부산역에서 가까운...,이색적까진 아니고 보통 이상 많이 가는 코스입니다!!참고하시고 가보실곳 있...
1383,2020.07.20,부산 여행코스,10시 조금넘어서 부산역에 도착해서 가까운곳 먼저갈려는데 유명하고 부산역에서 가까운...,부산역에서 제일 가까운 부산여행지는 부산역 맞은편 차이나타운과 초량이바구모...
1384,2020.07.20,부산 여행코스,10시 조금넘어서 부산역에 도착해서 가까운곳 먼저갈려는데 유명하고 부산역에서 가까운...,https://penguinworld9.tistory.com/290 ...
1385,2020.07.20,부산 여행코스,10시 조금넘어서 부산역에 도착해서 가까운곳 먼저갈려는데 유명하고 부산역에서 가까운...,부산 추천여행자료 참고적으로 보세요 .... 현재는 부산여행으로 이동이나 ...


In [5]:
df.to_csv(sch+'.csv', index=False,encoding='utf-8-sig')